#### By: Peyman Shahidi
#### Created: Oct 10, 2025

<br>

In [1]:
#Python
import getpass
import numpy as np
import pandas as pd
from collections import defaultdict
import itertools
import random 

## formatting number to appear comma separated and with two digits after decimal: e.g, 1000 shown as 1,000.00
pd.set_option('float_format', "{:,.2f}".format)

import matplotlib.pyplot as plt
#%matplotlib inline
#from matplotlib.legend import Legend

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 200)

In [2]:
main_folder_path = ".."
input_data_path = f"{main_folder_path}/data"
output_data_path = f'{input_data_path}/computed_objects'
output_plot_path = f"{main_folder_path}/writeup/plots"

In [3]:
# Create directories if they don't exist
import os

for path in [output_data_path, output_plot_path]:
    if not os.path.exists(path):
        os.makedirs(path)

### O*NET Data Processing

In [4]:
# Read all datasets
task_ratings_df = pd.read_csv(f'{input_data_path}/db_27_3_text/Task Ratings.txt', sep='\t')
task_categories_df = pd.read_csv(f'{input_data_path}/db_27_3_text/Task Categories.txt', sep='\t')
tasks_to_dwas_df = pd.read_csv(f'{input_data_path}/db_27_3_text/Tasks to DWAs.txt', sep='\t')
dwa_reference_df = pd.read_csv(f'{input_data_path}/db_27_3_text/DWA Reference.txt', sep='\t')
job_zones_df = pd.read_csv(f'{input_data_path}/db_27_3_text/Job Zones.txt', sep='\t')
task_statements_df = pd.read_csv(f'{input_data_path}/db_27_3_text/Task Statements.txt', sep='\t')
occupation_data_df = pd.read_csv(f'{input_data_path}/db_27_3_text/Occupation Data.txt', sep='\t')
soc_structure_df = pd.read_csv(f'{input_data_path}/SOC_Structure.csv')

In [5]:
# Merge and transform O*NET task data
ONET = task_ratings_df.merge(task_categories_df, on=['Scale ID', 'Category'], how='left')

# Process Category Description
ONET['Category Description'] = ONET['Category Description'].apply(lambda x: f'FT_{x}' if pd.notna(x) else x)
ONET['Category Description'] = ONET['Category Description'].fillna(ONET['Scale ID'])
ONET['Category Description'] = ONET['Category Description'].replace({'IM': 'Importance', 'RT': 'Relevance'})

# Reshape from long to wide format
ONET = ONET.pivot_table(
    index=['O*NET-SOC Code', 'Task ID'],
    columns='Category Description',
    values='Data Value',
    aggfunc='first'
).reset_index()
ONET.columns.name = None

# Merge additional data
ONET = ONET.merge(occupation_data_df[['O*NET-SOC Code', 'Title']], on='O*NET-SOC Code', how='left')
ONET.rename(columns={'Title': 'Occupation Title'}, inplace=True)

ONET = ONET.merge(task_statements_df[['O*NET-SOC Code', 'Task ID', 'Task', 'Task Type']], on=['O*NET-SOC Code', 'Task ID'], how='left')
ONET.rename(columns={'Task': 'Task Title'}, inplace=True)

ONET = ONET.merge(tasks_to_dwas_df[['O*NET-SOC Code', 'Task ID', 'DWA ID']], on=['O*NET-SOC Code', 'Task ID'], how='left')
ONET = ONET.merge(dwa_reference_df[['DWA ID', 'DWA Title']], on='DWA ID', how='left')
ONET = ONET.merge(job_zones_df[['O*NET-SOC Code', 'Job Zone']], on='O*NET-SOC Code', how='left')

### Filter Occupations Containing "All Other" and "Teachers"

In [6]:
# Remove rows where occupation title contains "All Other"
print(f"Number of rows before removing 'All Other': {ONET.shape[0]:,}")
ONET = ONET[~ONET["Occupation Title"].str.contains("All Other", case=False, na=False)]
print(f"Number of rows after removing 'All Other': {ONET.shape[0]:,}")

Number of rows before removing 'All Other': 22,310
Number of rows after removing 'All Other': 22,310


In [7]:
# # Filter and remove "Teachers"-related occupations
# contains_teacher = ONET[ONET['Occupation Title'].str.contains('Teachers', case=False, na=False)]

# unique_teacher_occupations = contains_teacher['Occupation Title'].nunique()
# print(f'Number of unique occupations containing the word "Teachers": {unique_teacher_occupations}')

# # Remove rows that contain "Teacher" (case-insensitive)
# ONET = ONET[~ONET['Occupation Title'].str.contains('Teachers', case=False, na=False)].reset_index(drop=True)
# print(f"Rows after removing Teachers: {len(ONET):,}")

### SOC Industry Structure

In [8]:
# Create SOC Code-to-Label mapping
code_label_rows = []
for idx, row in soc_structure_df.iterrows():
    if pd.notna(row['Major Group']):
        code = row['Major Group']
    elif pd.notna(row['Minor Group']):
        code = row['Minor Group']
    elif pd.notna(row['Broad Occupation']):
        code = row['Broad Occupation']
    elif pd.notna(row['Detailed Occupation']):
        code = row['Detailed Occupation']
    elif pd.notna(row['Detailed O*NET-SOC']):
        code = row['Detailed O*NET-SOC']
    else:
        continue
    code_label_rows.append({'Code': code, 'Label': row['SOC or O*NET-SOC 2019 Title']})

soc_code_label = pd.DataFrame(code_label_rows)
soc_code_label.to_csv(f'{output_data_path}/SOC_Code_Label_Mapping.csv', index=False)

In [9]:
# Create industry codes at different hierarchy levels
ONET['SOC_Code_7digit'] = ONET['O*NET-SOC Code'].str.split('.').str[0]

industry_levels = {
    2: 'Major_Group',
    5: 'Minor_Group', 
    6: 'Broad_Occupation',
    7: 'Detailed_Occupation'
}

for num_digits, level_name in industry_levels.items():
    if num_digits == 2:
        ONET[f'{level_name}_Code'] = ONET['SOC_Code_7digit'].str[:2] + '-0000'
    elif num_digits == 5:
        ONET[f'{level_name}_Code'] = ONET['SOC_Code_7digit'].str[:5] + '00'
    elif num_digits == 6:
        ONET[f'{level_name}_Code'] = ONET['SOC_Code_7digit'].str[:6] + '0'
    else:
        ONET[f'{level_name}_Code'] = ONET['SOC_Code_7digit']

# Drop SOC_Code_7digit from columns
ONET = ONET.drop(columns=['SOC_Code_7digit'])

In [10]:
# Add title labels for each industry level
for num_digits, level_name in industry_levels.items():
    code_col = f'{level_name}_Code'
    label_col = f'{level_name}_Title'
    ONET = ONET.merge(
        soc_code_label.rename(columns={'Code': code_col, 'Label': label_col}),
        on=code_col,
        how='left'
    )


# Aggregate unique task and DWA counts at each hierarchical level
def aggregate_by_level(df, code_col, title_col, level_name):
    """
    Aggregate unique task and DWA counts for a given hierarchical level.
    
    Parameters:
    - df: DataFrame to aggregate
    - code_col: Column name for the code/ID
    - title_col: Column name for the title/description
    - level_name: Name of the hierarchical level (e.g., 'Major Group')
    
    Returns:
    - DataFrame with columns: Level, Code, Title, num_unique_tasks, num_unique_dwas
    """
    agg = (
        df.groupby([code_col, title_col])
        .agg(
            num_unique_tasks=('Task ID', 'nunique'),
            num_unique_dwas=('DWA ID', 'nunique') if 'DWA ID' in df.columns else ('Task ID', lambda x: 0)
        )
        .reset_index()
        .rename(columns={code_col: 'Code', title_col: 'Title'})
    )
    agg['Level'] = level_name
    return agg[['Level', 'Code', 'Title', 'num_unique_tasks', 'num_unique_dwas']]


# Create aggregations for each hierarchical level
major_group_agg = aggregate_by_level(ONET, 'Major_Group_Code', 'Major_Group_Title', 'Major Group')
minor_group_agg = aggregate_by_level(ONET, 'Minor_Group_Code', 'Minor_Group_Title', 'Minor Group')
broad_occ_agg = aggregate_by_level(ONET, 'Broad_Occupation_Code', 'Broad_Occupation_Title', 'Broad Occupation')
detailed_occ_agg = aggregate_by_level(ONET, 'Detailed_Occupation_Code', 'Detailed_Occupation_Title', 'Detailed Occupation')
occupation_agg = aggregate_by_level(ONET, 'O*NET-SOC Code', 'Occupation Title', 'Occupation')

# Combine all levels into one dataset
hierarchical_task_counts = pd.concat([
    major_group_agg,
    minor_group_agg,
    broad_occ_agg,
    detailed_occ_agg,
    occupation_agg
], ignore_index=True)

# Save the combined dataset
hierarchical_task_counts.to_csv(f'{output_data_path}/hierarchical_task_counts.csv', index=False)

print(f"\nHierarchical Task Counts Summary:")
print(f"  Total rows: {len(hierarchical_task_counts):,}")
print(f"\nBreakdown by aggregation level:")
for level in ['Major Group', 'Minor Group', 'Broad Occupation', 'Detailed Occupation', 'Occupation']:
    count = len(hierarchical_task_counts[hierarchical_task_counts['Level'] == level])
    print(f"  {level}: {count}")


Hierarchical Task Counts Summary:
  Total rows: 2,179

Breakdown by aggregation level:
  Major Group: 22
  Minor Group: 95
  Broad Occupation: 430
  Detailed Occupation: 759
  Occupation: 873


In [11]:
# Check for duplicates with DWA ID and DWA Title
dup_cols_full = ['O*NET-SOC Code', 'Task ID', 'Task Type', 'DWA ID', 'DWA Title']
num_duplicates_full = ONET.duplicated(subset=dup_cols_full).sum()
print(f"Number of duplicate rows (with DWA ID & Title): {num_duplicates_full}")

# Check for duplicates without DWA ID and DWA Title
dup_cols_task = ['O*NET-SOC Code', 'Task ID', 'Task Type']
num_duplicates_task = ONET.duplicated(subset=dup_cols_task).sum()
print(f"Number of duplicate rows (without DWA ID & Title): {num_duplicates_task}")

# Show how many tasks are matched to multiple DWAs
task_counts = ONET.groupby(dup_cols_task)['DWA ID'].nunique()
multi_dwa_tasks = (task_counts > 1).sum()
print(f"Number of tasks matched to multiple DWAs: {multi_dwa_tasks}")

# Remove duplicates (keep first occurrence)
num_duplicates = ONET.duplicated(subset=dup_cols_full).sum()
ONET = ONET.drop_duplicates(subset=dup_cols_full).reset_index(drop=True)
print(f"Number of duplicate rows: {num_duplicates}")

# Reorder columns before saving
first_cols = ['O*NET-SOC Code', 'Occupation Title', 'Task ID', 'Task Title', 'Task Type', 'DWA ID', 'DWA Title', 'Job Zone']
last_cols = ['Major_Group_Code', 'Major_Group_Title', 'Minor_Group_Code', 'Minor_Group_Title', 
             'Broad_Occupation_Code', 'Broad_Occupation_Title', 'Detailed_Occupation_Code', 'Detailed_Occupation_Title']
middle_cols = [col for col in ONET.columns if col not in first_cols + last_cols]
ONET = ONET[first_cols + middle_cols + last_cols]

# Save final dataset
ONET.to_csv(f'{output_data_path}/ONET_cleaned_tasks.csv', index=False)

Number of duplicate rows (with DWA ID & Title): 0
Number of duplicate rows (without DWA ID & Title): 4357
Number of tasks matched to multiple DWAs: 3780
Number of duplicate rows: 0


In [12]:
# Create DWA repetition dataset across hierarchical levels
dwa_repetition_data = []

# Filter out rows where DWA ID is null
onet_with_dwa = ONET[ONET['DWA ID'].notna()]

for dwa_id in onet_with_dwa['DWA ID'].unique():
    dwa_data = onet_with_dwa[onet_with_dwa['DWA ID'] == dwa_id]
    dwa_title = dwa_data['DWA Title'].iloc[0]
    
    dwa_repetition_data.append({
        'DWA ID': dwa_id,
        'DWA Title': dwa_title,
        'num_occupations': dwa_data['O*NET-SOC Code'].nunique(),
        'num_detailed_occupations': dwa_data['Detailed_Occupation_Code'].nunique(),
        'num_broad_occupations': dwa_data['Broad_Occupation_Code'].nunique(),
        'num_minor_groups': dwa_data['Minor_Group_Code'].nunique(),
        'num_major_groups': dwa_data['Major_Group_Code'].nunique()
    })

dwa_repetition_df = pd.DataFrame(dwa_repetition_data)
dwa_repetition_df.to_csv(f'{output_data_path}/dwa_repetition_by_hierarchy.csv', index=False)

In [13]:
# # Create task repetition dataset across hierarchical levels
# task_repetition_data = []

# for task_id in ONET['Task ID'].unique():
#     task_data = ONET[ONET['Task ID'] == task_id]
    
#     # Get task title (should be consistent for same task ID)
#     task_title = task_data['Task Title'].iloc[0]
    
#     # Count occurrences at each hierarchical level
#     task_repetition_data.append({
#         'Task ID': task_id,
#         'Task Title': task_title,
#         'num_occupations': task_data['O*NET-SOC Code'].nunique(),
#         'num_detailed_occupations': task_data['Detailed_Occupation_Code'].nunique(),
#         'num_broad_occupations': task_data['Broad_Occupation_Code'].nunique(),
#         'num_minor_groups': task_data['Minor_Group_Code'].nunique(),
#         'num_major_groups': task_data['Major_Group_Code'].nunique()
#     })

# task_repetition_df = pd.DataFrame(task_repetition_data)

# # Save task repetition dataset
# task_repetition_df.to_csv(f'{output_data_path}/task_repetition_by_hierarchy.csv', index=False)

# print("Task Repetition Analysis:")
# print(f"  Total unique tasks: {len(task_repetition_df):,}")
# print(f"\nTasks appearing in multiple occupations:")
# print(f"  Tasks in 2+ occupations: {(task_repetition_df['num_occupations'] >= 2).sum():,}")
# print(f"  Tasks in 5+ occupations: {(task_repetition_df['num_occupations'] >= 5).sum():,}")
# print(f"  Tasks in 10+ occupations: {(task_repetition_df['num_occupations'] >= 10).sum():,}")
# print(f"\nMax repetition across levels:")
# print(f"  Max occupations per task: {task_repetition_df['num_occupations'].max()}")
# print(f"  Max detailed occupations per task: {task_repetition_df['num_detailed_occupations'].max()}")
# print(f"  Max broad occupations per task: {task_repetition_df['num_broad_occupations'].max()}")
# print(f"  Max minor groups per task: {task_repetition_df['num_minor_groups'].max()}")
# print(f"  Max major groups per task: {task_repetition_df['num_major_groups'].max()}")
# print(f"\nSaved to: {output_data_path}/task_repetition_by_hierarchy.csv")

# task_repetition_df.head(10)

# # Tasks are unique! ==> All 1s

In [14]:
# Sanity check: NaN values
nan_counts = ONET.isna().sum()
nan_summary = nan_counts[nan_counts > 0].sort_values(ascending=False)

print(f"Dataset shape: {ONET.shape}")
print(f"Total rows: {len(ONET):,}\n")

if len(nan_summary) > 0:
    print("Columns with NaN values:")
    print("=" * 50)
    for col, count in nan_summary.items():
        percentage = (count / len(ONET)) * 100
        print(f"{col}: {count:,} ({percentage:.2f}%)")
else:
    print("✓ No NaN values found in any column!")

Dataset shape: (22310, 25)
Total rows: 22,310

Columns with NaN values:
Minor_Group_Title: 4,231 (18.96%)
DWA ID: 425 (1.90%)
DWA Title: 425 (1.90%)
Broad_Occupation_Title: 215 (0.96%)
